In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter
import optuna as op

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [6]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 153 ( 0.4%) Users that have less than 1 test interactions


In [8]:
from libs.Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

In [9]:
def objective_function_IALS(optuna_trial):
    # Define the parameters to be tuned
    num_factors = optuna_trial.suggest_int("num_factors", 10, 200)  # Number of latent factors
    confidence_scaling = optuna_trial.suggest_categorical("confidence_scaling", ["linear", "log"])
    alpha = optuna_trial.suggest_float("alpha", 0.1, 2.0, step=0.1)  # Confidence weight
    epsilon = optuna_trial.suggest_float("epsilon", 0.1, 10.0, step=0.1)  # Used for log scaling only
    reg = optuna_trial.suggest_loguniform("reg", 1e-5, 1e-1)  # Regularization
    init_mean = optuna_trial.suggest_float("init_mean", -0.1, 0.1)  # Initialization mean
    init_std = optuna_trial.suggest_float("init_std", 0.01, 0.2)  # Initialization std
    
    # Instantiate and fit the recommender
    recommender_instance = IALSRecommender(URM_train)
    recommender_instance.fit(
        epochs=50,  # Fixed number of epochs, adjust if necessary
        num_factors=num_factors,
        confidence_scaling=confidence_scaling,
        alpha=alpha,
        epsilon=epsilon,
        reg=reg,
        init_mean=init_mean,
        init_std=init_std,
    )
    
    # Evaluate the recommender on the validation set
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    # Return the MAP at 10 for optimization
    return eval_res["MAP"][10]


In [10]:
import pandas as pd

class SaveResultsToCSV:
    def __init__(self, file_path=None):
        self.file_path = file_path
        self.results = []

    def __call__(self, study, trial):
        # Collect parameters and the resulting metric
        trial_result = trial.params.copy()
        trial_result["iteration"] = trial.number
        trial_result["MAP_value"] = trial.value
        self.results.append(trial_result)
        
        # Convert to DataFrame and save to CSV
        results_df = pd.DataFrame(self.results)
        results_df.to_csv(self.file_path, index=False)


In [13]:
# Instantiate the callback with the desired file path
save_results_to_csv = SaveResultsToCSV(file_path="result_experiments/IALS/results_optuna.csv")

# Create and optimize the Optuna study
optuna_study_IALS = op.create_study(direction="maximize")

optuna_study_IALS.optimize(
    objective_function_IALS,
    callbacks=[save_results_to_csv],
    n_trials=300  # Adjust as needed
)

# Check the results saved in the CSV file
print("Optimization results saved to 'result_experiments/IALS/results_optuna.csv'")

[I 2024-12-11 01:00:15,605] A new study created in memory with name: no-name-b06170ae-215c-49df-8364-dfb076f6da70
/var/folders/tr/d1rx7zt15lzgkp2yc1nwbg100000gn/T/ipykernel_31963/163794949.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  reg = optuna_trial.suggest_loguniform("reg", 1e-5, 1e-1)  # Regularization


IALSRecommender: Epoch 1 of 50. Elapsed time 11.30 min


[W 2024-12-11 01:11:54,085] Trial 0 failed with parameters: {'num_factors': 59, 'confidence_scaling': 'log', 'alpha': 0.7000000000000001, 'epsilon': 6.8, 'reg': 0.004209572203717478, 'init_mean': 0.050782753989069646, 'init_std': 0.04862937969547563} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/miniconda3/envs/RecSys/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/tr/d1rx7zt15lzgkp2yc1nwbg100000gn/T/ipykernel_31963/163794949.py", line 13, in objective_function_IALS
    recommender_instance.fit(
  File "/Users/safuan/Python/RecSysCompetition2024Polimi/libs/Recommenders/MatrixFactorization/IALSRecommender.py", line 88, in fit
    self._train_with_early_stopping(epochs,
  File "/Users/safuan/Python/RecSysCompetition2024Polimi/libs/Recommenders/Incremental_Training_Early_Stopping.py", line 199, in _train_with_early

KeyboardInterrupt: 